In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import re # regular expression library
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
def normalize(text):
    # Replace with whitespace to separate '😃\n\nFor'
    text = text.replace(r"\n", r" ")
    text = text.replace(r"\r", r" ")
    # Drop puntuation
    text = re.sub(r'[^\w\s]', '', text)
    # Remove extra spaces from '😃  For' to '😃 For'
    text = re.sub(r"\s+", r" ", text)
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove leading and trailing whitespace
    text = text.strip()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    #lower case the all capital alphabets
    text=text.lower()
    stop_words = set(stopwords.words('english'))
    word_tokens=word_tokenize(text)
    text = ' '.join([word for word in word_tokens if word not in stop_words])
    return text


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SINISTER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\SINISTER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer=Tokenizer()

def tok_vectorizer(text):
  test_sequences = tokenizer.texts_to_sequences(text)
  X_test_padded = pad_sequences(test_sequences, maxlen=35)
  return X_test_padded

In [123]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

In [124]:
from tensorflow.keras.models import load_model

model = load_model("text_based_emotion_classifier.h5")

In [125]:
text="getting scared"

In [126]:
x=normalize(text)
x=tok_vectorizer(x)

In [127]:
a=model.predict(x)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step


In [128]:
a[0]

array([0.1712919 , 0.16859983, 0.1610882 , 0.17817102, 0.16448042,
       0.15636863], dtype=float32)

In [138]:
# mood=sadness (0), joy (1), love (2), anger (3), fear (4),lazy(5)
mood=np.argmax(a[0],axis=0)
mood

3

In [152]:
mood_dict={0:'sadness',1:'joy',2:'love',3:'anger',4:'fear',5:'lazy'}
text_mood=str(mood_dict[mood])
text_mood

'anger'

In [154]:
mood_to_genre = {
    'sad': 'Drama',
    'joy': 'Comedy',
    'love': 'Romance',
    'anger': 'Action',
    'fear': 'Horror',
    'lazy': 'Relaxation'
}
genre=str(mood_to_genre[text_mood])
genre

'Action'

In [155]:
#recommender
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the dataset
df = pd.read_csv("movie_dataset.csv")

# Fill missing values for the specified features
features = ['keywords', 'cast', 'genres', 'director']
for feature in features:
    df[feature] = df[feature].fillna('')

# Combine features into a single string
def combined_features(row):
    return row['keywords'] + " " + row['cast'] + " " + row['genres'] + " " + row['director']

df['combined_features'] = df.apply(combined_features, axis=1)

# Create a CountVectorizer and transform the combined features
cv = CountVectorizer()
count_matrix = cv.fit_transform(df['combined_features'])

# Calculate cosine similarity
cosine_sim = cosine_similarity(count_matrix)

def recommend_movies(genre, title, top_n=10):
    # Check if the movie title is in the dataset
    if title not in df['title'].values:
        raise ValueError("Movie title not found in dataset")
    
    # Get the index of the movie with the given title
    def get_index_from(title):
        return df[df['title'] == title].index.values[0]

    movie_index = get_index_from(title)
    
    # Get similarity scores
    similar_movies = list(enumerate(cosine_sim[movie_index]))
    
    # Sort movies based on similarity score
    sorted_similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)
    
    # Function to get movie title from index
    def get_title_from_index(index):
        return df.iloc[index]['title']
    
    # Recommend movies of the specified genre
    recommendations = []
    for movie in sorted_similar_movies:
        idx = movie[0]
        if len(recommendations) >= top_n:
            break
        if df.iloc[idx]['genres'] == genre and df.iloc[idx]['title'] != title:
            recommendations.append(df.iloc[idx]['title'])
    
    return recommendations



In [159]:
title = input('enter title: ')
print('your current mood is',text_mood)

enter title: Avatar
your current mood is anger


In [160]:
recommended_movies = recommend_movies(genre, title)
print(f"Recommended movies similar to '{title}' in genre '{genre}':")
print(recommended_movies)

Recommended movies similar to 'Avatar' in genre 'Action':
['Furious 7', 'Once Upon a Time in Mexico', 'Stealth', 'Excessive Force', 'Only the Strong', 'Pound of Flesh', 'Alatriste', 'Out of Inferno', 'State of Play', 'Max Payne']
